In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/housesalesprediction/kc_house_data.csv


In [2]:
pip install pyspark

     |████████████████████████████████| 204.2 MB 30 kB/s 
     |████████████████████████████████| 198 kB 62.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612244 sha256=99a1bdf705152d3cec0b7ec54a5849f79ed8cf34db3fffa999f7977317cc9211
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
#Load Module
from pyspark.sql.types import *
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

In [4]:
#Creat Spark Session
spark=SparkSession.builder.master('local').appName('Regression House Price').getOrCreate()

In [5]:
#Load dataset
data=spark.read.csv('../input/housesalesprediction/kc_house_data.csv',inferSchema=True, header=True)
data.show(5)

+----------+---------------+--------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|        id|           date|   price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|grade|sqft_above|sqft_basement|yr_built|yr_renovated|zipcode|    lat|    long|sqft_living15|sqft_lot15|
+----------+---------------+--------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|7129300520|20141013T000000|221900.0|       3|      1.0|       1180|    5650|   1.0|         0|   0|        3|    7|      1180|            0|    1955|           0|  98178|47.5112|-122.257|         1340|      5650|
|6414100192|20141209T000000|538000.0|       3|     2.25|       2570|    7242|   2.0|         0|   0|        3|    7|      2170|          400|   

In [6]:
data.printSchema()

root
 |-- id: long (nullable = true)
 |-- date: string (nullable = true)
 |-- price: double (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: double (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- condition: integer (nullable = true)
 |-- grade: integer (nullable = true)
 |-- sqft_above: integer (nullable = true)
 |-- sqft_basement: integer (nullable = true)
 |-- yr_built: integer (nullable = true)
 |-- yr_renovated: integer (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- sqft_living15: integer (nullable = true)
 |-- sqft_lot15: integer (nullable = true)



In [7]:
df=data.select("price",'bedrooms','bathrooms','sqft_living','sqft_lot','floors','sqft_above','sqft_basement','sqft_living15','sqft_lot15')
data.show(5)

+----------+---------------+--------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|        id|           date|   price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|grade|sqft_above|sqft_basement|yr_built|yr_renovated|zipcode|    lat|    long|sqft_living15|sqft_lot15|
+----------+---------------+--------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|7129300520|20141013T000000|221900.0|       3|      1.0|       1180|    5650|   1.0|         0|   0|        3|    7|      1180|            0|    1955|           0|  98178|47.5112|-122.257|         1340|      5650|
|6414100192|20141209T000000|538000.0|       3|     2.25|       2570|    7242|   2.0|         0|   0|        3|    7|      2170|          400|   

In [8]:
df = df.withColumn("price", df["price"].cast(IntegerType()))
df = df.withColumn("bathrooms", df["bathrooms"].cast(IntegerType()))
df = df.withColumn("floors", df["floors"].cast(IntegerType()))
df.show(5)

+------+--------+---------+-----------+--------+------+----------+-------------+-------------+----------+
| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|sqft_above|sqft_basement|sqft_living15|sqft_lot15|
+------+--------+---------+-----------+--------+------+----------+-------------+-------------+----------+
|221900|       3|        1|       1180|    5650|     1|      1180|            0|         1340|      5650|
|538000|       3|        2|       2570|    7242|     2|      2170|          400|         1690|      7639|
|180000|       2|        1|        770|   10000|     1|       770|            0|         2720|      8062|
|604000|       4|        3|       1960|    5000|     1|      1050|          910|         1360|      5000|
|510000|       3|        2|       1680|    8080|     1|      1680|            0|         1800|      7503|
+------+--------+---------+-----------+--------+------+----------+-------------+-------------+----------+
only showing top 5 rows



In [9]:
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
price,21613,540088.1417665294,367127.19648270035,75000,7700000
bedrooms,21613,3.37084162309721,0.930061831147451,0,33
bathrooms,21613,1.7497339564151206,0.7348730839295018,0,8
sqft_living,21613,2079.8997362698374,918.4408970468096,290,13540
sqft_lot,21613,15106.967565816869,41420.51151513551,520,1651359
floors,21613,1.4462129274048026,0.5518935204355683,1,3
sqft_above,21613,1788.3906907879516,828.0909776519175,290,9410
sqft_basement,21613,291.5090454818859,442.57504267746685,0,4820
sqft_living15,21613,1986.552491556008,685.3913042527788,399,6210


In [10]:
#Split dataset
splitdata=df.randomSplit([0.8,0.2])
train=splitdata[0]
test=splitdata[1]

print('train:\n', train.count(), '\ntest:\n',test.count())

train:
 17287 
test:
 4326


In [11]:
df.printSchema()

root
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- sqft_above: integer (nullable = true)
 |-- sqft_basement: integer (nullable = true)
 |-- sqft_living15: integer (nullable = true)
 |-- sqft_lot15: integer (nullable = true)



In [12]:
#Assembler
assembler=VectorAssembler(inputCols=("price",'bedrooms','bathrooms','sqft_living','sqft_lot','floors','sqft_above','sqft_basement','sqft_living15','sqft_lot15'),
                         outputCol=('features'))
trainingdatasetfinal=assembler.transform(train).select(col('features'),(col('price').cast('Int').alias('Price')))
trainingdatasetfinal.show(5)


+--------------------+-----+
|            features|Price|
+--------------------+-----+
|[75000.0,1.0,0.0,...|75000|
|[78000.0,2.0,1.0,...|78000|
|[81000.0,2.0,1.0,...|81000|
|[82500.0,2.0,1.0,...|82500|
|[83000.0,2.0,1.0,...|83000|
+--------------------+-----+
only showing top 5 rows



In [13]:
#Modelling
algoritma = LinearRegression(
    labelCol="Price",featuresCol="features", maxIter=10, regParam=0.3)
#mentraining model kita dengan training data final
model = algoritma.fit(trainingdatasetfinal)
print ("Model Trained")

Model Trained


In [14]:
#Preparing Data Testing
assembler=VectorAssembler(inputCols=["price",'bedrooms','bathrooms','sqft_living','sqft_lot','floors','sqft_above','sqft_basement','sqft_living15','sqft_lot15'],outputCol='features')
testingdatasetfinal=assembler.transform(test).select(col('features'),(col('price').cast('Int').alias("Price")))
testingdatasetfinal.show(5)

+--------------------+------+
|            features| Price|
+--------------------+------+
|[80000.0,1.0,0.0,...| 80000|
|[82000.0,3.0,1.0,...| 82000|
|[92000.0,2.0,1.0,...| 92000|
|[100000.0,4.0,1.0...|100000|
|[109000.0,2.0,0.0...|109000|
+--------------------+------+
only showing top 5 rows



In [15]:
# Testing Data
data_pred=model.transform(testingdatasetfinal)
data_pred.show(5)

+--------------------+------+------------------+
|            features| Price|        prediction|
+--------------------+------+------------------+
|[80000.0,1.0,0.0,...| 80000| 80000.11210336909|
|[82000.0,3.0,1.0,...| 82000|  82000.1439377204|
|[92000.0,2.0,1.0,...| 92000| 92000.17891341654|
|[100000.0,4.0,1.0...|100000|100000.14707554577|
|[109000.0,2.0,0.0...|109000|109000.07693606177|
+--------------------+------+------------------+
only showing top 5 rows



In [16]:
#mengimpor modul untuk mengevaluasi akurasi regresi kita
from pyspark.ml.evaluation import RegressionEvaluator
#mendefinisikan evaluator dengan menginputkan kolom dari "label" 
#dan "prediksi" data kita, serta matrik evaluasi yg kita inginkan (rmse)
evaluator = RegressionEvaluator(
    labelCol="Price", predictionCol="prediction", metricName="rmse")
#menghitung rmse dg evalutor yang telah kita definisikan
rmse = evaluator.evaluate(data_pred)
print ("Root Mean Square Error (RMSE):", rmse)

Root Mean Square Error (RMSE): 0.451294750800467


 Let's upvote! Thanks.